<a href="https://colab.research.google.com/github/AlexBaunti/Prediction_AI/blob/main/RandomTreeMethod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Метод решающих деревьев

**Инициализация библиотек**

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np # обращение сокращенно np
import pandas as pd # обращение сокращенно pd
import matplotlib.pyplot as plt # обращение сокращенно plt
%matplotlib inline
from sklearn import linear_model, ensemble # алгоритмы sklearn
from sklearn.metrics import mean_absolute_error, mean_squared_error # меры качества

**Загрузка данных**

In [ ]:
%%capture
!wget https://raw.githubusercontent.com/AlexBaunti/Prediction_AI/main/COVID_Testing_Data.xlsx
!wget https://raw.githubusercontent.com/AlexBaunti/Prediction_AI/main/COVID_Training_Data.xlsx
training_data = pd.read_excel('COVID_Training_Data.xlsx')

Проверка данных - первые 5 строк таблицы с тренировочными данными

In [ ]:
training_data.head()

,Пробиотик,Возраст,Пол,Рост,Вес,ИМТ,ПлощадьТела,ПредраспССЗ,Курение,Алкоголь,...,МочКислот,Ддимеры,вчСРБ,ХЛНП,ИнАтероген,неХЛВП,ШкБорга,ТестEQ,ТШХдо,ТШХпосле
0,0,54,2,151,58,25.437481,1.559736,0,0,0,...,330,120,12.0,3.500000,4.875000,3.90,2,11,300,410
1,0,63,2,165,115,42.240588,2.295830,1,0,0,...,240,85,6.5,4.631818,4.500000,5.40,2,13,280,376
2,0,50,2,159,46,18.195483,1.425365,0,0,0,...,248,100,40.0,3.220909,6.306667,4.73,2,12,300,350
3,0,41,2,166,92,33.386558,2.059666,1,0,0,...,220,125,25.0,2.136364,2.676768,2.65,2,12,300,400
4,0,37,2,161,71,27.390919,1.781931,1,0,0,...,340,177,12.0,2.931818,3.700000,3.70,2,13,300,400


Считывание размерности таблицы с тренировочными данными

In [ ]:
training_data.shape

(64, 84)

**Проверка данных на пропуски и типы переменных**

Общая информация о данных

In [ ]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 84 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Пробиотик       64 non-null     int64  
 1   Возраст         64 non-null     int64  
 2   Пол             64 non-null     int64  
 3   Рост            64 non-null     int64  
 4   Вес             64 non-null     int64  
 5   ИМТ             64 non-null     float64
 6   ПлощадьТела     64 non-null     float64
 7   ПредраспССЗ     64 non-null     int64  
 8   Курение         64 non-null     int64  
 9   Алкоголь        64 non-null     int64  
 10  Гиподинамия     64 non-null     int64  
 11  КТ              64 non-null     int64  
 12  ДеньПослеCovid  64 non-null     int64  
 13  СлабостьДо      64 non-null     int64  
 14  БольСердцаДо    64 non-null     int64  
 15  Седцебиение     64 non-null     int64  
 16  ПеребоиДо       64 non-null     int64  
 17  ОдышкаДо        64 non-null     int64

Анализируем результата выполнения команды:

* 64 экземпляра (entries)
* 84 столбцов (columns)

Имеется всего три различных типа данных (dtypes):

* int64 - целое число - 47 столбцов;
* float64 - дробное число - 35 столбцов;
* object - объект (текст) - 2 столбца;

Цифры в каждой строчке обозначают количество заполненных (non-null) значений. Так как эти цифры не в каждой строчке совпадают с числом строк (64), то в данных существуют пропуски, которые необходимо учитывать.

**Разделение входных и выходной переменных**

Необходимо спрогнозировать эффективность выздоровление пациента, поэтому целевая переменная - это результат теста шестиминутной ходьбы. Запишем название столбца таблицы, в котором содержится целевая функция, в переменную target_variable_name:

In [ ]:
target_variable_name = 'ТШХпосле'

Теперь выделим в отдельную переменную training_target_variable весь столбец со значениями целевой переменной.

In [ ]:
training_target_variable = training_data[target_variable_name]
training_target_variable

0     410
1     376
2     350
3     400
4     400
     ... 
59    420
60    450
61    500
62    500
63    470
Name: ТШХпосле, Length: 64, dtype: int64

В отдельную переменную training_samples запишем признаки - то есть всю обучающую таблицу, за исключением целевой переменной. Для этого вызовем у переменной training_data метод drop() и удалим столбец "ТШХпосле".

In [ ]:
training_samples = training_data.drop(target_variable_name, axis=1)

Проверим, что столбец действительно удалён с помощью методов *head* и *shape*.

In [ ]:
training_samples.head()

,Пробиотик,Возраст,Пол,Рост,Вес,ИМТ,ПлощадьТела,ПредраспССЗ,Курение,Алкоголь,...,МНОтн,МочКислот,Ддимеры,вчСРБ,ХЛНП,ИнАтероген,неХЛВП,ШкБорга,ТестEQ,ТШХдо
0,0,54,2,151,58,25.437481,1.559736,0,0,0,...,0.87,330,120,12.0,3.500000,4.875000,3.90,2,11,300
1,0,63,2,165,115,42.240588,2.295830,1,0,0,...,0.98,240,85,6.5,4.631818,4.500000,5.40,2,13,280
2,0,50,2,159,46,18.195483,1.425365,0,0,0,...,1.01,248,100,40.0,3.220909,6.306667,4.73,2,12,300
3,0,41,2,166,92,33.386558,2.059666,1,0,0,...,0.90,220,125,25.0,2.136364,2.676768,2.65,2,12,300
4,0,37,2,161,71,27.390919,1.781931,1,0,0,...,0.96,340,177,12.0,2.931818,3.700000,3.70,2,13,300


In [ ]:
training_samples.shape

(64, 83)

Также удалим столбец с данными высокочувствительного С-реактивного белка (вчСРБ). По информации в нём заполнено лишь 15 из 64 значений, так что он не может учитываться при прогнозировании.

In [ ]:
training_samples = training_samples.drop('вчСРБ', axis=1)

Проверка данных

In [ ]:
training_samples.shape

(64, 82)

**Инициализация решающего алгоритма**

Лес решающих деревьев - ансамбль из множества решающих деревьев. Решающее дерево - это алгоритм, иерархически раскладывающий данные на множество бинарных условий. 

Для создания модели случайного леса вызываем модуль ensemble, затем через точку указываем название алгоритма RandomForestRegressor.

In [ ]:
random_forest_model = ensemble.RandomForestRegressor()
random_forest_model.get_params()

Вызываем метод fit() у модели и передаем ему на вход два аргумента: таблицу входных признаков и столбец значений целевой переменной - (training_samples, training_target_variable)

In [ ]:
random_forest_model.fit(training_samples, training_target_variable)